In [ ]:
import pandas as pd
from google.colab import files
import io

In [ ]:
uploaded = files.upload()

Saving traindata.csv to traindata.csv


In [ ]:
data = io.BytesIO(uploaded["traindata.csv"])
df = pd.read_csv(data, sep = None, engine = 'python')
df

,id,x1,x2,x3,y
0,1,60,64,0,1
1,2,54,60,11,0
2,3,65,62,22,0
3,4,34,60,0,1
4,5,38,69,21,0
...,...,...,...,...,...
292,293,65,67,0,1
293,294,53,65,12,0
294,295,57,64,1,0
295,296,54,59,7,1


In [ ]:
df.describe()

,id,x1,x2,x3,y
count,297.000000,297.000000,297.000000,297.000000,297.000000
mean,148.996633,52.468013,62.868687,4.121212,0.737374
std,85.874929,10.878311,3.236134,7.281418,0.440804
min,1.000000,30.000000,58.000000,0.000000,0.000000
25%,75.000000,44.000000,60.000000,0.000000,0.000000
50%,149.000000,52.000000,63.000000,1.000000,1.000000
75%,223.000000,61.000000,65.000000,5.000000,1.000000
max,296.000000,83.000000,69.000000,52.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      297 non-null    int64
 1   x1      297 non-null    int64
 2   x2      297 non-null    int64
 3   x3      297 non-null    int64
 4   y       297 non-null    int64
dtypes: int64(5)
memory usage: 11.7 KB


#Data Preprocessing


In [ ]:
#Split data untuk memisahkan features data dan target data
#data fitur
features = df[['x1', 'x2', 'x3']]
#data target
target = df["y"]

features.head()

,x1,x2,x3
0,60,64,0
1,54,60,11
2,65,62,22
3,34,60,0
4,38,69,21


#Normalisasi
###mengubah jarak data dari 0-1 agar jarak tidak terlalu timpang


In [ ]:
df = (df - df.min()) / (df.max()-df.min())

In [ ]:
features.describe()

,x1,x2,x3
count,297.000000,297.000000,297.000000
mean,52.468013,62.868687,4.121212
std,10.878311,3.236134,7.281418
min,30.000000,58.000000,0.000000
25%,44.000000,60.000000,0.000000
50%,52.000000,63.000000,1.000000
75%,61.000000,65.000000,5.000000
max,83.000000,69.000000,52.000000


In [ ]:
features.iloc[0] - features.iloc[1]

x1     6
x2     4
x3   -11
dtype: int64

#Function Distance
2 rumus yang dapat dipakai untuk menghitung jarak

In [ ]:
def euclidean_distance(x, y):
    dist = 0
    for xi, yi in zip(x, y):
        dist += (xi - yi)**2

    return dist ** 0.5

    return dist ** (1/2)

import numpy as np

def manhattan_distance(x, y):
    return np.abs(np.array(x) - np.array(y)).sum()

In [ ]:
print((features.iloc[0], features.iloc[1]))
print(euclidean_distance(features.iloc[0], features.iloc[1]))

(x1    60
x2    64
x3     0
Name: 0, dtype: int64, x1    54
x2    60
x3    11
Name: 1, dtype: int64)
13.152946437965905


In [ ]:
def predict(x,k,X,y):
  distance = []
  for _, row in X.iterrows():
    distance.append(manhattan_distance(x,row))

  data = X.copy()
  data['distance'] = distance
  data['clas'] = y
  data.sort_values(by="distance").reset_index(drop=True)

  y_pred = data.iloc[:k].clas.mode().values

  return y_pred[0]

In [ ]:
import numpy as np

def akurasi(y_pred, y_true):
    return np.mean(y_pred == y_true) * 100

#Folding
memisahkan dataframe menjadi beberapa bagian untuk melihat karakteristik data

In [ ]:
# Menentukan ukuran setiap bagian
size = len(df)
size_per_fold = size // 3

# Mengambil bagian pertama untuk testing dan dua bagian sisanya untuk training
fold_first = (df.iloc[:size_per_fold].reset_index(drop=True), df.iloc[size_per_fold:].reset_index(drop=True))

# Mengambil 50 persen data dengan menyilang masing-masing untuk training dan testing
fold_second = (pd.concat([df.iloc[:size_per_fold], df.iloc[2*size_per_fold:]]).reset_index(drop=True),
               pd.concat([df.iloc[size_per_fold:2*size_per_fold]]).reset_index(drop=True))

# Mengambil quarter akhir untuk testing dan dua quarter awal untuk training
fold_third = (df.iloc[size_per_fold:].reset_index(drop=True), df.iloc[:2*size_per_fold].reset_index(drop=True))

In [ ]:
accurate = []

for k in range(1,22,2):
    for fold in [fold_first, fold_second, fold_third]:
        train, test = fold
        X_train, y_train = train.drop(['y', 'id'], axis=1), train.y
        X_test, y_test = test.drop(['y', 'id'], axis=1), test.y

        y_preds = []
        for _, x in X_test.iterrows():
            y_preds.append(predict(x, k, X_train, y_train))

        accurate.append(akurasi(y_preds, y_test))

    print(f"Accuracy for k={k}: {(max(accurate)):.2f}%")

Accuracy for k=1: 78.79%
Accuracy for k=3: 78.79%
Accuracy for k=5: 78.79%
Accuracy for k=7: 78.79%
Accuracy for k=9: 78.79%
Accuracy for k=11: 78.79%
Accuracy for k=13: 78.79%
Accuracy for k=15: 78.79%
Accuracy for k=17: 78.79%
Accuracy for k=19: 78.79%
Accuracy for k=21: 78.79%
